## Transfer Learning

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/BoostcampAITech/lecture-note-python-basics-for-ai/blob/main/codes/pytorch/07_torch-study/transfer_learning/transfer_learning.ipynb)

In [ ]:
!pip install torchsummary

## Saving and Loading Model 

https://pytorch.org/tutorials/beginner/saving_loading_models.html

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import os

In [ ]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(3 * 3 * 64, 1000)
        self.fc2 = nn.Linear(1000, 1)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
# Initialize model
model = TheModelClass().cuda()

# Initialize optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [ ]:
type(model.state_dict())

In [ ]:
from torchsummary import summary
summary(model, (3, 224, 224))

In [ ]:
MODEL_PATH ="saved"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
torch.save(model.state_dict(), 
           os.path.join(MODEL_PATH, "model.pt"))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = TheModelClass()
new_model.load_state_dict(torch.load(os.path.join(
    MODEL_PATH, "model.pt")))

In [ ]:
torch.save(model, os.path.join(MODEL_PATH, "model_pickle.pt"))
model = torch.load(os.path.join(MODEL_PATH, "model_pickle.pt"))
model.eval()

In [ ]:
import zipfile

filename = "kagglecatsanddogs_3367a.zip"
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall()

import shutil
shutil.move('PetImages', 'data')

import os
from os import walk

mypath = "data"

f_path = []
for (dirpath, dirnames, filenames) in walk(mypath):
    f_path.extend([os.path.join(dirpath, filename) for filename in filenames])

from PIL import Image

for f_name in f_path:
    try:
        Image.open(f_name)
    except Exception as e:
        print(e)
        os.remove(f_name)


In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch

dataset = datasets.ImageFolder(root="data",
                           transform=transforms.Compose([
                               transforms.Scale(244),       
                               transforms.CenterCrop(244),  
                               transforms.ToTensor(),       
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                    (0.5, 0.5, 0.5)), 
                           ]))
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=8,
                                         shuffle=True,
                                         num_workers=8)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.1

In [ ]:
model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        
        optimizer.zero_grad()        
        y_pred = model(X_batch)
               
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        
    torch.save({
        'epoch': e,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
        }, f"saved/checkpoint_model_{e}_{epoch_loss/len(dataloader)}_{epoch_acc/len(dataloader)}.pt")
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

## Pretrained Model Loading

In [ ]:
import torch
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vgg = models.vgg16(pretrained=True).to(device)
print(vgg)

In [ ]:
from torchsummary import summary
summary(vgg, (3, 224, 224))

In [ ]:
for name, layer in vgg.named_modules():
    print(name, layer)

In [ ]:
vgg.fc = torch.nn.Linear(1000, 1)
vgg.cuda()

In [ ]:
vgg = models.vgg16(pretrained=True).to(device)
print(vgg)

In [ ]:
vgg.classifier._modules['6'] = torch.nn.Linear(4096, 1)
vgg.cuda()


## Pretrained Model

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch

dataset = datasets.ImageFolder(root="data/",
                           transform=transforms.Compose([
                               transforms.Scale(244),       
                               transforms.CenterCrop(244),  
                               transforms.ToTensor(),       
                               transforms.Normalize((0.5, 0.5, 0.5),
                                                    (0.5, 0.5, 0.5)), 
                           ]))
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=2,
                                         shuffle=True,
                                         num_workers=8)

In [ ]:
from torch import nn
from torchvision import models

class MyNewNet(nn.Module):   
    def __init__(self):
        super(MyNewNet, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        self.linear_layers = nn.Linear(1000, 1)


    # Defining the forward pass    
    def forward(self, x):
        x = self.vgg19(x)        
        return self.linear_layers(x)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

my_model = MyNewNet()
my_model = my_model.to(device)

In [ ]:
print(my_model)

In [ ]:
for param in my_model.parameters():
    param.requires_grad = False

In [ ]:
for param in my_model.linear_layers.parameters():
    param.requires_grad = True

In [ ]:
my_model.eval()

In [ ]:
x = next(iter(dataloader))

In [ ]:
x[0].shape

In [ ]:
my_model(x[0].to(device))

In [ ]:
x[1]

In [ ]:
EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 0.001

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True,
                                         num_workers=8)

In [ ]:
next(my_model.vgg19.features._modules['0'].parameters())[0]

In [ ]:
it = my_model.linear_layers.parameters()
next(it)[0][:10]

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

my_model = MyNewNet()
my_model = my_model.to(device)

for param in my_model.parameters():
    param.requires_grad = False
for param in my_model.linear_layers.parameters():
    param.requires_grad = True

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(my_model.parameters(), lr=LEARNING_RATE)


for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).type(torch.cuda.FloatTensor)
        
        optimizer.zero_grad()        
        y_pred = my_model(X_batch)
               
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(dataloader):.5f} | Acc: {epoch_acc/len(dataloader):.3f}')

In [ ]:
next(my_model.vgg19.features._modules['0'].parameters())[0]

In [ ]:
it = my_model.linear_layers.parameters()
next(it)[0][:10]

In [ ]:
tensor([-0.0109,  0.0245, -0.0065, -0.0201, -0.0019,  0.0256,  0.0250, -0.0139,
         0.0005, -0.0002], device='cuda:0', grad_fn=<SliceBackward>)
